In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

In [ ]:
import importlib
import yaml, itertools
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio

import pandas as pd
import random

import numpy as np

import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import os
import glob

import seaborn as sns


from sklearn.metrics import balanced_accuracy_score, confusion_matrix

In [ ]:
def make_color_list(emo_list):
    color = {'neu':'slategrey', 'ang':'crimson', 'hap':'gold', 
             'sad':'darkblue',  'dis':'plum', 
             'sur':'steelblue', 'fea':'olivedrab'}
    
    color_list = []
    for e in emo_list:
        c = color[e[:3]]
        color_list.append(c)
        
    return color_list

def make_center_color(color_list):
    color = {'slategrey':'black', 'crimson':'orangered', 'gold':'yellow', 
             'darkblue':'blue',  'plum':'purple', 
             'steelblue':'dodgerblue', 'olivedrab':'darkolivegreen'}
    
    center_list = []
    for e in color_list:
        c = color[e]
        center_list.append(c)
        
    return center_list

In [ ]:
class IEMOCAP_Dataset(Dataset):
    def __init__(self, data, hparams):
        
        self.hparams = hparams
        self.csv = data
        self.output_class = hparams['use_class']
        self.vad = hparams['vad']
        self.sr = 16000

        self.aranged_id_num = list(range(1,11))
        self.aranged_id_num.remove(2*hparams['fold_num']-1)
        self.aranged_id_num.remove(2*hparams['fold_num'])
    
    def __len__(self):
        return len(self.csv)

    def __getitem__(self, idx):

        data = self.csv.iloc[idx]
        ids = data['session']
        ans = self.output_class.index(data['emotion'])

        try:
            pid = self.aranged_id_num.index(data['id_num'])
        except:
            if data['id_num'] % 2 == 0:
                pid = 8
            else:
                pid = 9

        if self.vad:
            wav_path = '/media/ubuntu/SSD2/Dataset/IEMOCAP_VAD/Session'+str(data['fold'])\
                        +'/' + '_'.join(ids.split('_')[:-1]) + '/' + ids +'.wav'
        else:
            wav_path = '/media/ubuntu/SSD2/Dataset/IEMOCAP_full_release/Session'+str(data['fold'])\
                         +'/sentences/wav/' + '_'.join(ids.split('_')[:-1]) + '/' + ids +'.wav'
            if os.getcwd().split('/')[2] == 'cvnar2':
                wav_path = wav_path.replace('/media/ubuntu/SSD2/Dataset', '/home/cvnar2/Desktop/ssd')


        wav, sr = torchaudio.load(wav_path, normalize=True)
        if self.hparams['repeat_3_sec']:
            if wav.shape[-1] / self.sr < 3.0:
                n_repeat = int(3 // (wav.shape[-1] / self.sr))
                wav = wav.repeat((1, n_repeat))
                
        if (data['sec'] > self.hparams['max_sec']):
            max_len = int(16000 * self.hparams['max_sec'])
            offset = random.randint(0, wav.shape[1] - max_len - 1)
            wav = wav[:, offset:offset+max_len]

        inputs = wav.transpose(0, 1).squeeze(1)
        
        return inputs, ans, ids

In [ ]:
def Pad_Collate(samples):
    from torch.nn.utils.rnn import pad_sequence
    """
    DataLoader collate_fn
    """
    #for i in range(len(samples)):
    #    print(samples[i][0])
    #    print()
    inputs = [sample[0].squeeze(0) for sample in samples]  
    padded_inputs = pad_sequence(inputs, batch_first=True)

    labels = [sample[1] for sample in samples]
    labels = torch.Tensor(labels).float()
    labels = labels.unsqueeze(1)

    ids = [sample[2] for sample in samples]
        
    return padded_inputs, labels, ids

In [ ]:
main_path = 'weight/main_final2_load_emo/freeze'#'Vox_IEMO(3)/main_final2_load_emo_beta/all_finetune_beta(05)'
exp_list = os.listdir(main_path)
exp_list.sort()

bz = 1
if os.path.isdir(main_path + '/plot'):
    pass
else:
    os.mkdir(main_path + '/plot')

In [ ]:
exp_list

# plot T-sne

In [ ]:
exp = exp_list[0]

torch.cuda.empty_cache()

lib_path = glob.glob(main_path+'/'+exp+'/*main*.py')[0][:-3].replace('/', '.')
saved_main = importlib.import_module(lib_path)

u_path = glob.glob(main_path+'/'+exp+'/my_utils.py')[0][:-3].replace('/', '.')
my_utils = importlib.import_module(u_path)

import utils_data_10Fold

with open(main_path+'/'+exp+"/hparams.yaml") as f:
    hparams = yaml.load(f, Loader=yaml.FullLoader)

seed = hparams['seed']
my_utils.set_seed(seed)

print(hparams['repeat_3_sec'])

net = saved_main.Emotion_Network(hparams)
if hparams['id_net_freeze'] != 'freeze':
    net.id_net.hs.fc = nn.Parameter(torch.Tensor(8, hparams['fin_channel']))

net.id_filter.data = nn.Parameter(torch.Tensor(1, hparams['pool_head']+1, hparams['fin_channel']))

weight = torch.load(main_path+'/'+exp+"/best_model.pt")
missing_keys = net.load_state_dict(weight['model_state_dict'], strict=True)
print(missing_keys)

net = net.cuda()
net.eval()

trainset, validset, testset = utils_data_10Fold.real_5Fold(hparams)

test_set = IEMOCAP_Dataset(testset, hparams)
testloader = DataLoader(test_set, shuffle=False, batch_size=bz, collate_fn=Pad_Collate, pin_memory=True)

In [ ]:
d = next(iter(testloader))

outs = net.get_feat(d[0].cuda())

In [ ]:
plt.imshow(net.id_filter[0].data.clone().detach().cpu().numpy(), aspect='auto')
plt.colorbar()
plt.show()

In [ ]:
total_df = {'session': [], 'pred':[], 'mag':[], 'id_feat':[], 'ans':[], 'emo_feat':[]}
## emo: plot id: label 
for exp in exp_list:
    if exp == 'plot':
        continue
    print(exp)

    torch.cuda.empty_cache()

    lib_path = glob.glob(main_path+'/'+exp+'/*main*.py')[0][:-3].replace('/', '.')
    saved_main = importlib.import_module(lib_path)

    u_path = glob.glob(main_path+'/'+exp+'/my_utils.py')[0][:-3].replace('/', '.')
    my_utils = importlib.import_module(u_path)

    import utils_data_10Fold

    with open(main_path+'/'+exp+"/hparams.yaml") as f:
        hparams = yaml.load(f, Loader=yaml.FullLoader)

    seed = hparams['seed']
    my_utils.set_seed(seed)

    print(hparams['repeat_3_sec'])

    net = saved_main.Emotion_Network(hparams)
    if hparams['id_net_freeze'] != 'freeze':
        net.id_net.hs.fc = nn.Parameter(torch.Tensor(8, hparams['fin_channel']))

    net.id_filter.data = nn.Parameter(torch.Tensor(1, hparams['pool_head']+1, hparams['fin_channel']))

    weight = torch.load(main_path+'/'+exp+"/best_model.pt")
    missing_keys = net.load_state_dict(weight['model_state_dict'], strict=True)
    print(missing_keys)

    net = net.cuda()
    net.eval()
    
    trainset, validset, testset = utils_data_10Fold.real_5Fold(hparams)
    
    test_set = IEMOCAP_Dataset(testset, hparams)
    testloader = DataLoader(test_set, shuffle=False, batch_size=bz, collate_fn=Pad_Collate, pin_memory=True)

    """valid_set = IEMOCAP_Dataset(validset, hparams)
    validloader = DataLoader(valid_set, shuffle=True, batch_size=bz, collate_fn=saved_main.Pad_Collate, pin_memory=True)

    trainset = IEMOCAP_Dataset(trainset, hparams)
    trainloader = DataLoader(trainset, batch_size=bz, collate_fn=saved_main.Pad_Collate, pin_memory=True)"""

    # GE FEATURES
    full_inp = []
    emo_feat_list = []
    id_feat_list = []
    feat_list = []
    new_feat_list = []
    mag_list = []

    id_list = []
    ans_list = []
    loader_list = []
    out_list = []

    emo_list = hparams['use_class']
    color_list = make_color_list(emo_list)

    for idx, iemo_dataloader in enumerate([testloader]):

        name = "test"

        for wav, ans, pid in tqdm(iemo_dataloader):
            wav = wav.cuda()

            with torch.no_grad():
                out, feat, feat_list = net.get_feat(wav)

            emo_feat_list.extend(feat.detach().cpu().numpy())
            id_feat_list.extend(feat_list[0].detach().cpu().numpy())
            mag_list.append(torch.sum(feat[0].detach().cpu()**2).numpy())
            #feat_list.extend(feat.detach().cpu().numpy())
            #new_feat_list.extend(new_feat.detach().cpu().numpy())

            id_list.append(pid[0])
            ans_list.append(int(ans[0][0].item()))
            loader_list.append(name)
            out_list.append(out[0][0].cpu().argmax(dim=-1).numpy())
            

    id_list = np.array(id_list)
    ans_list = np.array(ans_list)
    loader_list = np.array(loader_list)
    mag_list = np.array(mag_list)

    emo_feat_list = np.array(emo_feat_list)
    id_feat_list = np.array(id_feat_list)
    out_list = np.array(out_list)
    #new_feat_list = np.array(new_feat_list)

    total_df['session'].extend(id_list) 
    total_df['pred'].extend(out_list)
    total_df['mag'].extend(mag_list)

    total_df['emo_feat'].extend(emo_feat_list)
    total_df['id_feat'].extend(id_feat_list)
    total_df['ans'].extend(ans_list)


In [ ]:
df = pd.DataFrame(total_df)

In [ ]:
r = pd.read_csv('result.csv')
r

In [ ]:
n_tmp = pd.read_csv('result.csv')
n_tmp = pd.concat((n_tmp, df['id_feat']), axis=1)
n_tmp

In [ ]:
cls_list = hparams['use_class']
color_list = make_color_list(cls_list)

tsne = TSNE(n_components=2, perplexity=20, init='pca', learning_rate='auto')
y_tsne = tsne.fit_transform(np.array(total_df['id_feat']))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16,8))
i = 0
for k in list(set(n_tmp['ans'])):
    mask2 = np.where(np.array(n_tmp['ans'])==k, True, False)

    x = y_tsne[mask2][:, 0]
    y = y_tsne[mask2][:, 1]
    ax[0].scatter(x, y, label=cls_list[i], color=color_list[i], alpha=0.5, s=10)
    i+=1

ax[0].legend()
ax[0].set_title(exp + ' ' + name)

# ID
i = 0
cmap1 = plt.cm.get_cmap('tab10', 10)
for k in list(set(n_tmp['id_num'])):
    mask2 = np.where(np.array(n_tmp['id_num'])==k, True, False)

    x = y_tsne[mask2][:, 0]
    y = y_tsne[mask2][:, 1]
    ax[1].scatter(x, y, label=k, color=cmap1(i), alpha=0.5, s=10)
    i+=1

ax[1].legend()
ax[1].set_title(exp + ' ' + name)

plt.tight_layout()
#plt.savefig(main_path + '/plot/'+exp+'_' +state + '_emo_id_repeat3sec.png')
plt.show()

In [ ]:
for cls, ang in zip(list(range(1,11)), np.arange(0, 360, 36)):
    print(cls, ang)

In [ ]:
set(n_tmp['id_num'])

In [ ]:
cls_list, color_list

In [ ]:
id_mag = np.sqrt(np.sum(np.array(total_df['id_feat'])**2, axis=1))

color_list = make_color_list(['ang', 'hapexc', 'sad', 'neu']) 
figure, axes = plt.subplots(1,1, figsize=(8,8))
import math
r = id_mag.max()

axes.set_aspect(1)
draw_circle = plt.Circle((0, 0), r, fill=False)
axes.add_artist(draw_circle)

#draw_circle = plt.Circle((0, 0), 1, fill=False, alpha=0.5)
#axes.add_artist(draw_circle)

#axes.set_xlim(-1.2*r,1.2*r)
#axes.set_ylim(-1.2*r,1.2*r)

idx = 0
cmap1 = plt.cm.get_cmap('tab10', 10)
for cls, ang in zip(list(range(1,11)), np.arange(0, 360, 36)):
    #axes.plot((0,math.cos(ang*math.pi/180)*r), (0, math.sin(ang*math.pi/180)*r), color=color_list[i], alpha=0.2)

    m1 = np.where(n_tmp['id_num'] == cls, True, False)
    i = 0
    for e in cls_list:
        m2 = np.where(n_tmp['emotion'] == e, True, False)
        m = m1 & m2

        tmp2 = id_mag[m]
        #p#rint(ang.shape, torch.tensor(tmp2).shape, torch.randint_like(torch.tensor(tmp2), -10, 10).shape)
        angle = ang + torch.randint_like(torch.tensor(tmp2), -15, 15)
        
        axes.scatter(torch.cos(angle*torch.pi/180)*tmp2, torch.sin(angle*torch.pi/180)*tmp2, c=color_list[i], s=4, alpha=0.7)
        i += 1

#tmp2 = tmp[tmp['cls'] == 'neu']['val']
#ang = torch.randint_like(torch.tensor(list(tmp2)), 0, 360)
#ax.scatter(torch.cos(ang*torch.pi/180)*tmp2, torch.sin(ang*torch.pi/180)*tmp2, c=color_list[idx], label='neu', s=8, alpha=0.7)

#axes.set_title(loader)
#axes.legend()
plt.show()

In [ ]:
id_mag = np.sqrt(np.sum(np.array(total_df['id_feat'])**2, axis=1))

color_list = make_color_list(['ang', 'hapexc', 'sad', 'neu']) 
figure, axes = plt.subplots(1,4, figsize=(16,4))
import math
r = id_mag.max()



#draw_circle = plt.Circle((0, 0), 1, fill=False, alpha=0.5)
#axes.add_artist(draw_circle)

#axes.set_xlim(-1.2*r,1.2*r)
#axes.set_ylim(-1.2*r,1.2*r)

idx = 0
i = 0
for e in cls_list:
    axes[idx].set_aspect(1)
    draw_circle = plt.Circle((0, 0), r, fill=False)
    axes[idx].add_artist(draw_circle)
    for cls, ang in zip(list(range(1,11)), np.arange(0, 360, 36)):
        m1 = np.where(n_tmp['id_num'] == cls, True, False)
        m2 = np.where(n_tmp['emotion'] == e, True, False)
        m = m1 & m2

        tmp2 = id_mag[m]
        #p#rint(ang.shape, torch.tensor(tmp2).shape, torch.randint_like(torch.tensor(tmp2), -10, 10).shape)
        angle = ang + torch.randint_like(torch.tensor(tmp2), -15, 15)
        
        axes[idx].scatter(torch.cos(angle*torch.pi/180)*tmp2, torch.sin(angle*torch.pi/180)*tmp2, c=color_list[idx], s=4, alpha=0.7)
        axes[idx].set_title(e)
    idx += 1

#tmp2 = tmp[tmp['cls'] == 'neu']['val']
#ang = torch.randint_like(torch.tensor(list(tmp2)), 0, 360)
#ax.scatter(torch.cos(ang*torch.pi/180)*tmp2, torch.sin(ang*torch.pi/180)*tmp2, c=color_list[idx], label='neu', s=8, alpha=0.7)

    
#axes.legend()
plt.tight_layout()
plt.show()

In [ ]:
id_mag = np.sqrt(np.sum(np.array(total_df['id_feat'])**2, axis=1))

color_list = make_color_list(['ang', 'hapexc', 'sad', 'neu']) 
figure, axes = plt.subplots(1,1, figsize=(8,8))
import math
r = id_mag.max()

axes.set_aspect(1)
draw_circle = plt.Circle((0, 0), r, fill=False)
axes.add_artist(draw_circle)

#draw_circle = plt.Circle((0, 0), 1, fill=False, alpha=0.5)
#axes.add_artist(draw_circle)

#axes.set_xlim(-1.2*r,1.2*r)
#axes.set_ylim(-1.2*r,1.2*r)

idx = 0
cmap1 = plt.cm.get_cmap('tab10', 10)
for cls, ang in zip(list(range(1,11)), np.arange(0, 360, 36)):
    axes.plot((0,math.cos(ang*math.pi/180)*r), (0, math.sin(ang*math.pi/180)*r), color=cmap1(idx), alpha=0.2)

    tmp2 = np.where(n_tmp['id_num'] == cls, True, False)
    tmp2 = id_mag[tmp2]

    ang += torch.randint_like(torch.tensor(list(tmp2)), -10, 10)
    
    axes.scatter(torch.cos(ang*torch.pi/180)*tmp2, torch.sin(ang*torch.pi/180)*tmp2, color=cmap1(idx), label=cls, s=8, alpha=0.7)
    idx += 1

#tmp2 = tmp[tmp['cls'] == 'neu']['val']
#ang = torch.randint_like(torch.tensor(list(tmp2)), 0, 360)
#ax.scatter(torch.cos(ang*torch.pi/180)*tmp2, torch.sin(ang*torch.pi/180)*tmp2, c=color_list[idx], label='neu', s=8, alpha=0.7)

#axes.set_title(loader)
axes.legend()
plt.show()

In [ ]:
n_tmp.to_csv('result.csv', index=False)

In [ ]:
csv = pd.read_csv('/home/ubuntu/Desktop/MY_GITHUB/VAD_net/emo_vad_sec.csv')
if hparams['concat_ExcHap']:
    csv = csv.replace('exc', 'hapexc')
    csv = csv.replace('hap', 'hapexc')

mask = [True if e in ['ang', 'hapexc', 'sad', 'neu'] else False for e in csv['emotion']]
csv = csv[mask]
csv = csv.reset_index(drop=True)

tmp = pd.concat((csv, df['pred']), axis=1)
emo_dict = {e: e_idx for e_idx, e in enumerate(emo_list)}

n_df = pd.DataFrame([emo_dict[val] for val in tmp['emotion']], columns=['ans'])
n_tmp = pd.concat((tmp, n_df), axis=1)
n_tmp

In [ ]:
n_tmp = pd.concat((n_tmp, df['mag']), axis=1)

m = np.where(n_tmp['pred'] == n_tmp['ans'], 'O', 'X')
correct_df = pd.DataFrame(m, columns=['correct'])

n_tmp = pd.concat((n_tmp, correct_df), axis=1)
n_tmp

In [ ]:
n_tmp.to_csv('result.csv', index=False)

In [ ]:
n_tmp = pd.read_csv('result.csv')
n_tmp['mag'] = np.sqrt(n_tmp['mag'])

color_list = make_color_list(['ang', 'hapexc', 'sad', 'neu']) 
figure, axes = plt.subplots(1,1, figsize=(8,8))
import math
r = 20
for loader in zip(['test']):
    axes.set_aspect(1)
    draw_circle = plt.Circle((0, 0), r, fill=False)
    axes.add_artist(draw_circle)

    #draw_circle = plt.Circle((0, 0), 1, fill=False, alpha=0.5)
    #axes.add_artist(draw_circle)

    #axes.set_xlim(-1.2*r,1.2*r)
    #axes.set_ylim(-1.2*r,1.2*r)

    idx = 0
    for cls, ang in zip(['ang', 'hapexc', 'sad', 'neu'], [0, 90, 180, 270]):
        axes.plot((0,math.cos(ang*math.pi/180)*r), (0, math.sin(ang*math.pi/180)*r), c=color_list[idx], label=cls, alpha=0.2)

        tmp2 = n_tmp[n_tmp['emotion'] == cls]['mag']

        ang += torch.randint_like(torch.tensor(list(tmp2)), -30, 30)
        
        axes.scatter(torch.cos(ang*torch.pi/180)*tmp2, torch.sin(ang*torch.pi/180)*tmp2, c=color_list[idx], label=cls, s=8, alpha=0.7)
        idx += 1

    #tmp2 = tmp[tmp['cls'] == 'neu']['val']
    #ang = torch.randint_like(torch.tensor(list(tmp2)), 0, 360)
    #ax.scatter(torch.cos(ang*torch.pi/180)*tmp2, torch.sin(ang*torch.pi/180)*tmp2, c=color_list[idx], label='neu', s=8, alpha=0.7)

    #axes.set_title(loader)
    axes.legend()
plt.show()

In [ ]:
import math 

figure, axes = plt.subplots(1,1, figsize=(8,8))

n_tmp['mag'] = (n_tmp['mag'] - n_tmp['mag'].min()) / (n_tmp['mag'].max() - n_tmp['mag'].min()) * 5
r = 5
for loader in zip(['test']):
    axes.set_aspect(1)
    draw_circle = plt.Circle((0, 0), r, fill=False)
    axes.add_artist(draw_circle)

    draw_circle = plt.Circle((0, 0), 1, fill=False, alpha=0.5)
    axes.add_artist(draw_circle)

    axes.set_xlim(-1.2*r,1.2*r)
    axes.set_ylim(-1.2*r,1.2*r)

    idx = 0
    for cls, ang in zip(['ang', 'hapexc', 'sad', 'neu'], [0, 90, 180, 270]):
        axes.plot((0,math.cos(ang*math.pi/180)*r), (0, math.sin(ang*math.pi/180)*r), c=color_list[idx], label=cls, alpha=0.2)

        tmp2 = n_tmp[n_tmp['emotion'] == cls]['mag']

        ang += torch.randint_like(torch.tensor(list(tmp2)), -30, 30)
        
        axes.scatter(torch.cos(ang*torch.pi/180)*tmp2, torch.sin(ang*torch.pi/180)*tmp2, c=color_list[idx], label=cls, s=8, alpha=0.7)
        idx += 1

    #tmp2 = tmp[tmp['cls'] == 'neu']['val']
    #ang = torch.randint_like(torch.tensor(list(tmp2)), 0, 360)
    #ax.scatter(torch.cos(ang*torch.pi/180)*tmp2, torch.sin(ang*torch.pi/180)*tmp2, c=color_list[idx], label='neu', s=8, alpha=0.7)

    #axes.set_title(loader)
    axes.legend()
plt.show()

In [ ]:
import math 

figure, axes = plt.subplots(1,1, figsize=(8,8))

n_tmp['mag'] = np.abs((n_tmp['mag'] - n_tmp['mag'].mean()) / n_tmp['mag'].std())
r =3
for loader in zip(['test']):
    axes.set_aspect(1)
    draw_circle = plt.Circle((0, 0), r, fill=False)
    axes.add_artist(draw_circle)

    draw_circle = plt.Circle((0, 0), 1, fill=False, alpha=0.5)
    axes.add_artist(draw_circle)

    axes.set_xlim(-1.2*r,1.2*r)
    axes.set_ylim(-1.2*r,1.2*r)

    idx = 0
    for cls, ang in zip(['ang', 'hapexc', 'sad', 'neu'], [0, 90, 180, 270]):
        axes.plot((0,math.cos(ang*math.pi/180)*r), (0, math.sin(ang*math.pi/180)*r), c=color_list[idx], label=cls, alpha=0.2)

        tmp2 = n_tmp[n_tmp['emotion'] == cls]['mag']

        ang += torch.randint_like(torch.tensor(list(tmp2)), -30, 30)
        
        axes.scatter(torch.cos(ang*torch.pi/180)*tmp2, torch.sin(ang*torch.pi/180)*tmp2, c=color_list[idx], label=cls, s=8, alpha=0.7)
        idx += 1

    #tmp2 = tmp[tmp['cls'] == 'neu']['val']
    #ang = torch.randint_like(torch.tensor(list(tmp2)), 0, 360)
    #ax.scatter(torch.cos(ang*torch.pi/180)*tmp2, torch.sin(ang*torch.pi/180)*tmp2, c=color_list[idx], label='neu', s=8, alpha=0.7)

    #axes.set_title(loader)
    axes.legend()
plt.show()

In [ ]:
for e in emo_list:
    sns.pairplot(n_tmp[n_tmp['emotion']==e])
    plt.title(e)
    plt.show()

In [ ]:
sns.jointplot(data=n_tmp, x='V', y='emotion', xlim=(1,5))

In [ ]:
for e in emo_list:
    sns.jointplot(data=n_tmp[n_tmp['emotion']==e], x='V', y='A', hue='correct', xlim=(1,5), ylim=(1,5)) #, hue="emotion")
    plt.title(e)
    plt.show()

In [ ]:
for e in emo_list:
    dd = n_tmp[n_tmp['emotion'] == e]
    correct = np.where(dd['pred'] == dd['ans'], True, False)

    acc = correct.sum() / len(correct) 
    print('{}:\t {:.4f}'.format(e, acc))

In [ ]:
for e in list(set(n_tmp['n_gender'])):
    dd = n_tmp[n_tmp['n_gender'] == e]
    correct = np.where(dd['pred'] == dd['ans'], True, False)

    acc = correct.sum() / len(correct) 
    print('{}:\t {:.4f}'.format(e, acc))

In [ ]:
for e in list(set(n_tmp['n_mode'])):
    dd = n_tmp[n_tmp['n_mode'] == e]
    correct = np.where(dd['pred'] == dd['ans'], True, False)

    acc = correct.sum() / len(correct) 
    print('{}:\t {:.4f}'.format(e, acc))

In [ ]:
neu_v_mean = n_tmp[n_tmp['emotion']=='neu']['V'].mean()
neu_a_mean = n_tmp[n_tmp['emotion']=='neu']['A'].mean()
neu_d_mean = n_tmp[n_tmp['emotion']=='neu']['D'].mean()

vv = (n_tmp['V'] - neu_v_mean)**2
aa = (n_tmp['A'] - neu_a_mean)**2
dd = (n_tmp['D'] - neu_d_mean)**2

iten = np.sqrt(vv+ aa+ dd)

iten.min(), iten.max()

In [ ]:
iten_df = pd.DataFrame((iten), columns=['intensity'])
n_tmp = pd.concat((n_tmp, iten_df), axis=1)

In [ ]:
iten = np.sqrt(vv+ aa)

iten_df = pd.DataFrame((iten), columns=['intensity2'])
n_tmp = pd.concat((n_tmp, iten_df), axis=1)

In [ ]:
sns.histplot(n_tmp, x='V', hue='emotion' ,multiple='dodge', bins=np.arange(0, 5, 0.5), shrink=0.8, stat='percent', common_norm=False, kde=True)

In [ ]:
sns.histplot(n_tmp, x='intensity', hue='emotion' ,multiple='dodge', bins=np.arange(0, 4, 0.1), shrink=0.8, stat='percent', common_norm=False, kde=True)

In [ ]:
sns.histplot(n_tmp, x='intensity2', hue='emotion' ,multiple='dodge', bins=np.arange(0, 4, 0.1), shrink=0.8, stat='percent', common_norm=False, kde=True)

In [ ]:
iten = np.sqrt(vv+ aa)

vv = (n_tmp['V'])**2
aa = (n_tmp['A'])**2
dd = (n_tmp['D'])**2

iten = np.sqrt(vv+aa+dd)
iten_df = pd.DataFrame((iten), columns=['intensity3'])
n_tmp = pd.concat((n_tmp, iten_df), axis=1)

iten = np.sqrt(vv+aa)
iten_df = pd.DataFrame((iten), columns=['intensity4'])
n_tmp = pd.concat((n_tmp, iten_df), axis=1)

In [ ]:
n_tmp['intensity4'].min() , n_tmp['intensity4'].max()

In [ ]:
sns.histplot(n_tmp, x='intensity3', hue='emotion' ,multiple='dodge', bins=np.arange(2, 9, 1), shrink=0.8, stat='percent', common_norm=False, kde=True)

In [ ]:
sns.histplot(n_tmp, x='intensity4', hue='emotion' ,multiple='dodge', bins=np.arange(1, 8, 1), shrink=0.8, stat='percent', common_norm=False, kde=True)

In [ ]:
sns.histplot(n_tmp, x='intensity3', hue='emotion' ,multiple='dodge', bins=np.arange(1, 8, 0.5), shrink=0.8, stat='percent', common_norm=False, kde=True)